# Higher-order functions

## Programming Fundamentals (NB17)

### MIEIC/2020-21

#### João Correia Lopes

INESC TEC, FEUP

## Goals

By the end of this class, the student should be able to:

- Describe the use higher-order functions
- Describe the use nested functions and closures
- Describe the use function *currying* and *uncurrying*
- Describe the use of partial function application
- Describe the use the operators available in Python

## Bibliography

- David Mertz, *Functional Programming in Python*, O'Reilly Media, 2015 
[[HTML]](https://www.oreilly.com/library/view/functional-programming-in/9781492048633/)
- Composing Programs, a free online introduction to programming and computer science (Section 1.6)
[[HTML]](https://composingprograms.com/pages/16-higher-order-functions.html)


# First-class functions

> When you say that a language has first-class functions, it means that the language treats functions as values:

- You can store the function in a variable
- You can pass the function as a parameter to another function
- You can return the function from a function
- You can store them in data structures such as hash tables, lists, …
- In Python a function is an instance of the Object type

### Functions as objects

A Python program to illustrate functions can be treated as objects:

In [ ]:
def shout(text):
    return text.upper() + "!"

print(shout('Hello'))

yell = shout
  
print(yell('Goodbye'))

## First-class vs Higher-order functions

### Higher-order functions

- To express certain general patterns as named concepts, we will need to construct functions that can accept other functions as arguments or return functions as values

- Functions that manipulate functions are called **higher-order functions**

- higher-order functions (HOF) can serve as powerful abstraction mechanisms, vastly increasing the expressive power of our language

### “higher-order” vs. “first-class”

- The “higher-order” (HOF) concept can be applied to functions in general, like functions in the mathematical sense

- The “first-class” concept only has to do with functions in programming languages:

  - It’s not used when referring to a function, such as “a first-class function”

  - It’s much more common to say that “a language has/hasn’t first-class function support”

- The two things are closely related, as it’s hard to imagine a language with first-class functions that would not also support higher-order functions, and conversely a language with higher-order functions but without first-class function support.

# Higher-order functions

## HOF

- We've already seen some built-in HOF:

  - `map()`, `filter()`

- We saw an iterator algebra that builds on the `itertools` module. 

- In some ways, HOF provide similar building blocks to express complex
concepts by combining simpler functions into new functions

### Sorting: An Example of HOF

- In order to define non-default sorting in Python, both the `sorted()` function and the list’s `sort()` method accept a key argument

- The value passed to this argument needs to be a function object that returns the sorting key for any item in the list or iterable

```
>>> def second_element(t):
...     return t[1]
...
>>> ledzep = [('Guitar', 'Jimmy'), ('Vocals', 'Robert'), ('Bass', 'John Paul'), ('Drums', 'John')]

>>> sorted(ledzep)
[('Bass', 'John Paul'), ('Drums', 'John'), ('Guitar', 'Jimmy'), ('Vocals', 'Robert')]

>>> sorted(ledzep, key=second_element)
[('Guitar', 'Jimmy'), ('Drums', 'John'), ('Bass', 'John Paul'), ('Vocals', 'Robert')]
```

## Functions as parameters

### sum_squares()

- Consider the functions `sum_squares()`, which computes the sum of the cubes of natural numbers up to `n`

In [ ]:
def sum_squares(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k*k, k + 1
    return total

print(sum_squares(100))

### sum_cubes()

- Consider the functions `sum_cubes()`, which computes the sum of the cubes of natural numbers up to `n`

In [ ]:
def sum_cubes(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k*k*k, k + 1
    return total

print(sum_cubes(100))

- `sum_squares()` and `sum_cubes()` clearly share a common underlying pattern
  - They are for the most part identical, differing only in name and the **function of k used to compute the term** to be added

### `summation()` HOF

- `summation()` takes as its two arguments the upper bound `n` together with the function `term` that computes the kth term

```
>>> def summation(n, term):
        total, k = 0, 1
        while k <= n:
            total, k = total + term(k), k + 1
        return total
```


In [ ]:
def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total

def sum_cubes(n):
    return summation(n, lambda x: x*x*x)

print(sum_cubes(3))

See how this works in [Python tutor](http://www.pythontutor.com/visualize.html#mode=edit)

### `sum_naturals()`

To sum the natural numbers we may use the identity function.

In [ ]:
def identity(x):
    return x

def sum_naturals(n):
    return summation(n, identity)

print(sum_naturals(100))

### Callbacks

- In a higher-order function, when one of the parameters passed in is a function, that function is a *callback function* because it will be called back and used within the higher-order function

- A higher-order function is named as such because when using a callback to perform an operation within itself, the function has a ‘higher’ purpose than a regular function

- When it returns a function, it also has a ‘higher’ purpose

## Functions as return values

### Nested functions

- A function which is defined inside another function is known as nested function

- Nested functions are able to access variables of the enclosing scope

- In Python, these non-local variables can be accessed only within their scope and not outside their scope

In [ ]:
def outer_function(text):
    itext = text

    def inner_function():  # nested function
        print(itext)       # accesses 'itext' as non-local variable
  
    inner_function()

outer_function('Hey!')

### Scope of variables

![images](images/17/scope-in-python.png)

$\Rightarrow$ 
https://www.datacamp.com/community/tutorials/scope-of-variables-python

In [ ]:
# Think about the scopes of x and y before running this code

x = 0
y = 3
def outer():
    x = 1
    def inner():
        x = 2
        print("inner x:", x)
        print("global y:", y)

    inner()
    print("outer x:", x)

outer()
print("global x:", x)

$\Rightarrow$ 
https://www.datacamp.com/community/tutorials/scope-of-variables-python

### A common mistake

- A common mistake is attempting to encapsulate an internal variable using an **immutable type** (`int`, `float`, `complex`, `string`, `tuple`, `frozen set`)

- When it is re-assigned in the inner scope, it is interpreted as a new variable and fails because it hasn’t been defined

In [ ]:
def outer():
    count = 0
    def inner():
        count = count + 1
        return count
    return inner

counter = outer()
print(counter())

The standard workaround for this issue is to use a mutable datatype (`list`, `dict`, `set`) and manage state within that object.

In [ ]:
def better_outer():
    count = [0]
    def inner():
        count[0] = count[0] + 1
        return count[0]
    return inner

counter = better_outer()
print(counter())
print(counter())
print(counter())

### Closures

- A closure is a way of keeping alive a variable even when the function has returned

- In a closure, a function is defined along with the environment

- In Python, this is done by nesting a function inside the encapsulating function and then returning the underlying function

```
def add_5():
    five = 5
    def add(arg):  # nesting functions
        return arg + five
    return add

    closure1 = add_5()
    print(closure1(1))  # output 6
    print(closure1(2))  # output 7
```

Another more general closure:

In [ ]:
def outer(a):
    def inner(b):
        return a + b
    return inner

add5 = outer(5)
print(add5)

print(add5(10))

### Example of using closures

```
import logging
logging.basicConfig(filename='example.log', level=logging.INFO)

def logger(func):
    def log_func(*args):
        logging.info(
            'Running "{}" with arguments {}'.format(func.__name__, args))
        return func(*args)
    return log_func    # returning WITHOUT parenthesis

def add(x, y): return x+y
add_logger = logger(add)
print(add_logger(3, 3))
print(add_logger(4, 5))
```


```
def sub(x, y): return x-y
sub_logger = logger(sub)
print(sub_logger(10, 5))
print(sub_logger(20, 10))
```

$\Rightarrow$ 
https://www.geeksforgeeks.org/python-closures/

## Utility Higher-Order Functions

### `compose()`

- A handy utility is `compose()` that takes a sequence of functions 
and returns a function that represents the application of 
each of these argument functions to a data argument:

In [ ]:
def compose(*funcs):
    """Return a new function compose(f,g,...)(x) == f(g(...(x)))."""
    def inner(data, funcs=funcs):
        result = data
        for f in reversed(funcs):
            result = f(result)
        return result
    return inner

In [ ]:
mod6 = lambda x: x%6
times2 = lambda x: x*2
minus3 = lambda x: x-3

f = compose(mod6, times2, minus3)

all(f(i)==((i-3)*2)%6 for i in range(1000000))

## Currying

### higher-order functions & currying

- We can use higher-order functions to convert a function that takes multiple arguments into a chain of functions that each take a single argument

- More specifically, given a function `f(x, y)`, we can define a function `g` such that `g(x)(y)` is equivalent to `f(x, y)`

- Here, `g` is a higher-order function that takes in a single argument `x` and returns another function that takes in a single argument `y`

- This transformation is called *currying*<sup>1</sup>

<sup>1</sup> Named after [Haskell Curry](https://en.wikipedia.org/wiki/Haskell_Curry)

- As an example, we can define a curried version of the `pow()` function:

In [ ]:
def curried_pow(x):
    def h(y):
        return pow(x, y)
    return h

curried_pow(2)(3)

- *Currying* is useful when we require a function that takes in only a single argument

- For example, the map pattern applies a single-argument function to a sequence of values

In [ ]:
def map_to_range(f, start, end):
    while start < end:
        print(f(start))
        start = start + 1

In [ ]:
map_to_range(curried_pow(2), 0, 10)

## The `functools` Module

### The `functools` Module HOF

- The `functools` module contains some higher-order functions

  - `partial()`, `add()`, `reduce()`, `sum()`

  - https://docs.python.org/3/library/functools.html

- we already saw `reduce()`

```
>>> import functools, operator
>>> functools.reduce(operator.add, [1, 2, 3, 4], 10)
20
>>> sum([1, 2, 3, 4])
10
```

  - `reduce()` gives the final result of `accumulate()` (from the `itertools` module: [PSL](https://docs.python.org/3/library/itertools.html?highlight=accum#itertools.accumulate))

### Partial function application

- The most useful tool in this module is the `functools.partial()` function

- For programs written in a functional style, you’ll sometimes want to construct variants of existing functions that have some of the parameters filled in

- Consider a Python function `f(a, b, c)`; you may wish to create a new function `g(b, c)` that’s equivalent to `f(1, b, c)`; you’re filling in a value for one of f()’s parameters

- This is called “partial function application”

- Currying is often confused with partial application

> Where partial application takes a function and from it builds a function which takes fewer arguments, currying builds functions which take multiple arguments by composition of functions which each take a single argument. 

$\Rightarrow$
https://en.wikipedia.org/wiki/Partial_application

```
import functools

def log(message, subsystem):
    """Write the contents of 'message' to the specified subsystem."""
    print('%s: %s' % (subsystem, message))
    ...

server_log = functools.partial(log, subsystem='server')
server_log('Unable to open socket')
```
$\Rightarrow$
https://docs.python.org/3/howto/functional.html#the-functools-module

In [ ]:
from functools import partial
basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

## The `operator` Module

### Useful Function Objects

- There are many builtin functions in Python that accept functions as arguments

- An example is the `filter()` function that was used previously

- However, there are some basic actions that use operators instead of functions (like `+` or the subscript `[]` or dot `.` operators)

- The `operator` module provides function versions of these operators

```
>>> import operator
>>> operator.add(1, 2)
3
```

$\Rightarrow$
https://docs.python.org/3/library/operator.html

Operator modules operations may replace the use of many lambda functions; for example `operator.concat`:

In [ ]:
import operator, functools
functools.reduce(operator.concat, ['A', 'BB', 'A'])

Operation `operator.itemgetter()` may also be used as key to sort collections:

In [ ]:
ledzep = [('Guitar', 'Jimmy'), ('Vocals', 'Robert'), ('Bass', 'John Paul'), ('Drums', 'John')]

sorted(ledzep)
[('Bass', 'John Paul'), ('Drums', 'John'), ('Guitar', 'Jimmy'), ('Vocals', 'Robert')]

In [ ]:
import operator

sorted(ledzep, key=operator.itemgetter(1))
[('Guitar', 'Jimmy'), ('Drums', 'John'), ('Bass', 'John Paul'), ('Vocals', 'Robert')]

$\Rightarrow$
https://www.protechtraining.com/content/python_fundamentals_tutorial-functional_programming

## Function decorators (bonus)

- Yes, there's more....

$\Rightarrow$ 
[Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/#simple-decorators)

$\Rightarrow$ 
[13.10. Decorators](https://www.protechtraining.com/content/python_fundamentals_tutorial-functional_programming)


# Ticket to leave

## Moodle activity

[LE17: Higher-order functions](https://moodle.up.pt/course/view.php?id=1738#section-1)


$\Rightarrow$ 
[Go back to the Table of Contents](00-contents.ipynb)

$\Rightarrow$ 
[Read the Preface](00-preface.ipynb)